In [1]:
import pandas as pd
import numpy as np
from jax import vmap
import jax.numpy as jnp
from jax.nn import softmax
import evofr as ef

In [2]:
# Getting data
raw_seq = pd.read_csv("../data/pango_location-variant-sequence-counts.tsv", sep="\t")
raw_seq = raw_seq[raw_seq.location == "USA"]
variant_frequencies = ef.VariantFrequencies(raw_seq)

_How do I set the baseline Pango lineage in the MLR model? For this analysis, I'd like all estimates to be relative to BA.2._

In [3]:
# Defining model
mlr = ef.MultinomialLogisticRegression(tau=4.2)

In [4]:
# Defining inference method
inference_method = ef.InferFullRank(iters=10_000, lr=0.01, num_samples=100)

In [5]:
# Fitting model
posterior = inference_method.fit(mlr, variant_frequencies)
samples = posterior.samples

In [6]:
def forecast_frequencies(samples, mlr, forecast_L):
    """
    Use posterior beta to forecast posterior frequenicies.
    """
    
    # Making feature matrix for forecasting
    last_T = samples["freq"].shape[1]
    X = mlr.make_ols_feature(start=last_T, stop=last_T + forecast_L)
    
    # Posterior beta
    beta = jnp.array(samples["beta"])
    
    # Matrix multiplication by sample
    dot_by_sample = vmap(jnp.dot, in_axes=(None, 0), out_axes=0)
    logits = dot_by_sample(X, beta) # Logit frequencies by variant
    return softmax(logits, axis=-1)

forecast_L = 30
samples["freq_forecast"] = forecast_frequencies(samples, mlr, forecast_L)

In [7]:
ga = samples["ga"]

_This has growth advantages for 141 Pango lineages as expected. However, I don't know what order these are in. I can't inspect `posterior` to get a list of variant labels._

In [8]:
ga.shape

(100, 141)

In [9]:
type(ga)

jaxlib.xla_extension.DeviceArray

In [10]:
posterior.data.var_names[:10]

['B',
 'B.1.1.529',
 'BA.1',
 'BA.1.1',
 'BA.1.1.1',
 'BA.1.1.10',
 'BA.1.1.14',
 'BA.1.1.16',
 'BA.1.1.18',
 'BA.1.1.2']

In [18]:
ga_df = pd.DataFrame(
    ef.posterior.get_growth_advantage(samples, posterior.data, ps=[0.8], name="USA", rel_to="BA.2")
)
ga_df.head(10)

,location,variant,median_ga,ga_upper_80,ga_lower_80
0,USA,B,0.278789,0.3080219,0.2386076
1,USA,B.1.1.529,1.3742096,1.4103048,1.3301693
2,USA,BA.1,0.60435945,0.67924964,0.5167244
3,USA,BA.1.1,0.6828772,0.69443613,0.6690454
4,USA,BA.1.1.1,0.6780826,0.7043556,0.65418255
5,USA,BA.1.1.10,0.15725043,0.21408884,0.10921383
6,USA,BA.1.1.14,0.67330897,0.6937073,0.65087426
7,USA,BA.1.1.16,0.45388535,0.6449719,0.3127773
8,USA,BA.1.1.18,0.6720482,0.70823705,0.63827485
9,USA,BA.1.1.2,0.6226268,0.6510542,0.5935625


In [19]:
ga_df.to_csv('growth_advantages.tsv', sep="\t")